## Instalacion de dependecias e importación de librerías

In [1]:
#pip install plotly_express

In [2]:
import pandas as pd
import plotly.express as px
from scipy import stats as st

## Importación y verificación de datos

In [3]:
df_companies = pd.read_csv("https://practicum-content.s3.us-west-1.amazonaws.com/learning-materials/data-analyst-eng/moved_project_sql_result_01.csv")
df_locations = pd.read_csv("https://practicum-content.s3.us-west-1.amazonaws.com/learning-materials/data-analyst-eng/moved_project_sql_result_04.csv")
df_hipotesis = pd.read_csv("https://practicum-content.s3.us-west-1.amazonaws.com/learning-materials/data-analyst-eng/moved_project_sql_result_07.csv")

In [4]:
df_companies.info

<bound method DataFrame.info of                                  company_name  trips_amount
0                                   Flash Cab         19558
1                   Taxi Affiliation Services         11422
2                            Medallion Leasin         10367
3                                  Yellow Cab          9888
4             Taxi Affiliation Service Yellow          9299
..                                        ...           ...
59               4053 - 40193 Adwar H. Nikola             7
60                    2733 - 74600 Benny Jona             7
61              5874 - 73628 Sergey Cab Corp.             5
62  2241 - 44667 - Felman Corp, Manuel Alonso             3
63                3556 - 36214 RC Andrews Cab             2

[64 rows x 2 columns]>

In [5]:
df_locations.info

<bound method DataFrame.info of    dropoff_location_name  average_trips
0                   Loop   10727.466667
1            River North    9523.666667
2          Streeterville    6664.666667
3              West Loop    5163.666667
4                 O'Hare    2546.900000
..                   ...            ...
89       Mount Greenwood       3.137931
90             Hegewisch       3.117647
91              Burnside       2.333333
92             East Side       1.961538
93             Riverdale       1.800000

[94 rows x 2 columns]>

In [6]:
df_hipotesis.info

<bound method DataFrame.info of                  start_ts weather_conditions  duration_seconds
0     2017-11-25 16:00:00               Good            2410.0
1     2017-11-25 14:00:00               Good            1920.0
2     2017-11-25 12:00:00               Good            1543.0
3     2017-11-04 10:00:00               Good            2512.0
4     2017-11-11 07:00:00               Good            1440.0
...                   ...                ...               ...
1063  2017-11-25 11:00:00               Good               0.0
1064  2017-11-11 10:00:00               Good            1318.0
1065  2017-11-11 13:00:00               Good            2100.0
1066  2017-11-11 08:00:00               Good            1380.0
1067  2017-11-04 16:00:00                Bad            2834.0

[1068 rows x 3 columns]>

Los datos parecen adecuados en todos sus campos para el analisis que deseamos realizar

## Análisis de datos

In [7]:
fig = px.bar(df_companies[0:9],x="company_name", y="trips_amount")
fig.show()
display(df_companies.head(10))

,company_name,trips_amount
0,Flash Cab,19558
1,Taxi Affiliation Services,11422
2,Medallion Leasin,10367
3,Yellow Cab,9888
4,Taxi Affiliation Service Yellow,9299
5,Chicago Carriage Cab Corp,9181
6,City Service,8448
7,Sun Taxi,7701
8,Star North Management LLC,7455
9,Blue Ribbon Taxi Association Inc.,5953


Tomando la tabla y el gráfico de barras como referencia, podemos notar que la empresa "Flash Cab" tiene una destacable cuota del mercado de taxis, siendo utilizada mas del doble de veces que la siguiente empresa de viajes

In [8]:
fig = px.bar(df_locations[0:9],x="dropoff_location_name", y="average_trips")
fig.show()
display(df_locations.head(10))

,dropoff_location_name,average_trips
0,Loop,10727.466667
1,River North,9523.666667
2,Streeterville,6664.666667
3,West Loop,5163.666667
4,O'Hare,2546.900000
5,Lake View,2420.966667
6,Grant Park,2068.533333
7,Museum Campus,1510.000000
8,Gold Coast,1364.233333
9,Sheffield & DePaul,1259.766667


In [ ]:

#Descargamos un Geojson con los datos geograficos de los barrios de chicago
#url = "https://data.cityofchicago.org/api/geospatial/bbvz-uum9?method=export&format=GeoJSON"
url = "./Boundaries - Neighborhoods.geojson"

#utilizamos plotly express para generar un mapbox con los datos de viajes por cada barrio
fig = px.choropleth_mapbox(df_locations, geojson=url, color="average_trips",
                           locations="dropoff_location_name", featureidkey="properties.pri_neigh",
                           center={"lat": 41.83, "lon":  -87.6298},
                           mapbox_style="carto-positron", zoom=9)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

Tomando ese mapbox como referencia, nos podemos dar cuenta que la mayoría de viajes dentro de la ciudad de Chicago tienen como destino el centro de la ciudad, a excepcion del barrio O´hare, del que, investigando un poco mas a fondo, nos dimos cuenta que es el aeropuerto internacional de Chicago.

## Prueba de hipotesis

La duración promedio de los viajes desde el Loop hasta el Aeropuerto Internacional O'Hare cambia los sábados lluviosos

In [10]:
fig = px.histogram(df_hipotesis, x="duration_seconds", color="weather_conditions")
fig.update_layout(barmode='overlay',bargap=0.015)

In [11]:
# Prueba de hipótesis
#H0: La duración promedio de los viajes desde el Loop hasta el Aeropuerto Internacional O'Hare los sábados lluviosos es el mismo
#H1: La duración promedio de los viajes desde el Loop hasta el Aeropuerto Internacional O'Hare los sábados lluviosos es diferente
#Como tenemos dos muestras que deseamos comparar, utilizaremos la Hipótesis sobre la igualdad de las medias de dos poblaciones.

buen_clima = df_hipotesis[df_hipotesis["weather_conditions"]=="Good"]["duration_seconds"]
mal_clima = df_hipotesis[df_hipotesis["weather_conditions"]=="Bad"]["duration_seconds"]

#Ejecutamos la prueba de studient independiente mandando como parámetros ambas muestras, y establecemos el valor de equal_var en False, ya que desconocemos la varianza de ambas muestras y debemos ser más consevadores en las conlcusiones
tstudent_test = st.ttest_ind(buen_clima,mal_clima,equal_var=False)
print("p-value:",tstudent_test.pvalue)

p-value: 6.738994326108734e-12


Con un valor de p significativamente mas pequeño que el nivel de signifiancia establecido(.05), podemos decir que existe evidencia estadística para afirmar que la duración promedio de los viajes desde el Loop hasta el Aeropuerto Internacional O'Hare es diferente en días con buen clima en comparación con días con mal clima.

## Conclusiones

En este analisis pudimos observar con claridad el comportamiento de los usuarios de taxis en la ciudad de Chicago, dandonos cuenta que la mayor parte de viajes se concentra en el centro de la ciudad y sus alrededores, ademas que la compañia de taxis líder en el mercado supera casi al doble a su siguiente competidor.

También, realizamos un experimento para comparar los viajes que se realizan en días con buen clima y mal clima, y pudimos llegar a la conclusión que la duración de los viajes sí es diferente al comparar ese factor.